<a href="https://colab.research.google.com/github/jesusGS01/TGINE/blob/main/4_1_TFIDF_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sesión 4.1 - Procesamiento de texto con la librería scikit-learn
En esta sesión de prácticas vamos a ver cómo podemo usar las funciones de tratamiento de texto de la librería scikit-learn para calcular TF, TF-IDF y BM25

Primero, instalaremos la librerías necesarias.

In [ ]:
!pip3 install -U scikit-learn
# Descargamos un fichero python con la implementación del BM25
!wget https://dis.um.es/~valencia/recursosTGINE/BM25.py

# Descargamos el fichero datasetEspañol.csv
!wget https://dis.um.es/~valencia/recursosTGINE/datasetEspañol.csv


In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
# Importamos del fichero BM25.py
from BM25 import BM25Transformer

In [4]:
# Leemos los datos del dataset en español de la primera sesión
data = pd.read_csv("datasetEspañol.csv",encoding="UTF-8")
data.tail()

,twitter_id,twitter_created_at,tweet,corpus,user,agreement,votes,score,label,__split
5954,1274324047581581316,2020-06-20 16:51:43,No me fío nada! ? en #estadodealarma demostrar...,Estado de alarma nacional (oficial),GuzmanitaMaria,100,1,-1,negative,test
5955,1274321386446733315,2020-06-20 16:41:08,@roldanfj1 @MikiyDuarte @diariocadiz @realDona...,Estado de alarma nacional (oficial),ByChanchi,100,1,-1,negative,train
5956,1274340519271858178,2020-06-20 17:57:10,"Con el fin del #EstadodeAlarma se acaban, tamb...",Estado de alarma nacional (oficial),Javiersilvestre,100,1,-1,negative,val
5957,1274367246979211269,2020-06-20 19:43:22,@horaciorlarreta. @AsisOberdan. @luisnovaresio...,Estado de alarma nacional (oficial),juliodebarna,100,1,-1,negative,train
5958,1274368625047220224,2020-06-20 19:48:51,En la última semana se han registrado 36 falle...,Estado de alarma nacional (oficial),mallorcadiario,100,1,-1,negative,train


## Apartado 1.1 - Obtener TF del conjunto de texto (Resuelto)

Calculamos la matriz de TF usando la clase CountVectorizer sobre un conjunto de textos.

Se puede consultar información de esta clase en la siguiente URL:
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer

Hay que tener en cuenta que esta clase tiene muchos parámetros en su método de creación.

In [5]:
texto = """La asignatura de TGINE es una asignatura del máster de BigData que se estudia en la Universidad de Murcia.
En la asignatura de máster vemos una introducción al procesamiento del lenguaje natural y tecnologías de procesamiento
de información no estructurada.
"""
texto2 = "No me gusta el chocolate ni las fresas"

texto3 = """El profesor de la asignatura TGINE es Rafael Valencia García.
"""
# Calculamos la matriz de TF usando la función fit_transform
count_vect = CountVectorizer()
X_counts = count_vect.fit_transform([texto,texto2,texto3])

# Mostramos entonces el número de textos y el número de tokens únicos
print(X_counts.shape)

# X_counts es una matriz dispersa con el TF de cada token en cada texto
# Imprimimos los textos y su correspondientes TF
print(texto)
print(X_counts[0])
print(texto2)
print(X_counts[1])
print(texto3)
print(X_counts[2])

#Los tokens de todo el vocabulario se representan con ids que hacen referencia a cada token.
print("Mostramos los items del diccionario")
print(count_vect.vocabulary_.items())
print("Tamaño vocabulario:", str(len(count_vect.vocabulary_.items())))

#Mostramos el código de una palabra determinada
#hay que tener en cuenta que todos los tokens se guardan en minúsculas
palabra_a_buscar="TGINE"
print("Código de la palabra", palabra_a_buscar, "es:", count_vect.vocabulary_.get(palabra_a_buscar.lower()))

(3, 36)
La asignatura de TGINE es una asignatura del máster de BigData que se estudia en la Universidad de Murcia.
En la asignatura de máster vemos una introducción al procesamiento del lenguaje natural y tecnologías de procesamiento
de información no estructurada.

  (0, 16)	3
  (0, 1)	3
  (0, 4)	6
  (0, 31)	1
  (0, 8)	1
  (0, 32)	2
  (0, 5)	2
  (0, 21)	2
  (0, 2)	1
  (0, 27)	1
  (0, 29)	1
  (0, 10)	1
  (0, 7)	2
  (0, 33)	1
  (0, 20)	1
  (0, 35)	1
  (0, 15)	1
  (0, 0)	1
  (0, 25)	2
  (0, 18)	1
  (0, 22)	1
  (0, 30)	1
  (0, 14)	1
  (0, 24)	1
  (0, 9)	1
No me gusta el chocolate ni las fresas
  (0, 24)	1
  (0, 19)	1
  (0, 13)	1
  (0, 6)	1
  (0, 3)	1
  (0, 23)	1
  (0, 17)	1
  (0, 11)	1
El profesor de la asignatura TGINE es Rafael Valencia García.

  (0, 16)	1
  (0, 1)	1
  (0, 4)	1
  (0, 31)	1
  (0, 8)	1
  (0, 6)	1
  (0, 26)	1
  (0, 28)	1
  (0, 34)	1
  (0, 12)	1
Mostramos los items del diccionario
dict_items([('la', 16), ('asignatura', 1), ('de', 4), ('tgine', 31), ('es', 8), ('una', 32), 

## Apartado 1.2 Calculamos el TF sin tener en cuenta las stopwords
Para eso hacemos uso del parámetro stop_words de CountVectorizer

In [8]:
import nltk
# Descargamos las stopwords de NLTK
# Si no tenemos instalado NLTK lo instalamos
# !pip3 install -U nltk
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('spanish')

# Calculamos la matriz de TF usando la función fit_transform
count_vect = CountVectorizer(stop_words=stopwords)
X_counts = count_vect.fit_transform([texto,texto2,texto3])

# Mostramos entonces el número de textos y el número de tokens únicos
print(X_counts.shape)

# X_counts es una matriz dispersa con el TF de cada token en cada texto
# Imprimimos los textos y su correspondientes TF
print(texto)
print(X_counts[0])
print(texto2)
print(X_counts[1])
print(texto3)
print(X_counts[2])

#Los tokens de todo el vocabulario se representan con ids que hacen referencia a cada token.
print("Mostramos los items del diccionario")
print(count_vect.vocabulary_.items())

(3, 22)
La asignatura de TGINE es una asignatura del máster de BigData que se estudia en la Universidad de Murcia.
En la asignatura de máster vemos una introducción al procesamiento del lenguaje natural y tecnologías de procesamiento
de información no estructurada.

  (0, 0)	3
  (0, 18)	1
  (0, 12)	2
  (0, 1)	1
  (0, 4)	1
  (0, 19)	1
  (0, 11)	1
  (0, 21)	1
  (0, 9)	1
  (0, 14)	2
  (0, 10)	1
  (0, 13)	1
  (0, 17)	1
  (0, 8)	1
  (0, 3)	1
No me gusta el chocolate ni las fresas
  (0, 7)	1
  (0, 2)	1
  (0, 5)	1
El profesor de la asignatura TGINE es Rafael Valencia García.

  (0, 0)	1
  (0, 18)	1
  (0, 15)	1
  (0, 16)	1
  (0, 20)	1
  (0, 6)	1
Mostramos los items del diccionario
dict_items([('asignatura', 0), ('tgine', 18), ('máster', 12), ('bigdata', 1), ('estudia', 4), ('universidad', 19), ('murcia', 11), ('vemos', 21), ('introducción', 9), ('procesamiento', 14), ('lenguaje', 10), ('natural', 13), ('tecnologías', 17), ('información', 8), ('estructurada', 3), ('gusta', 7), ('chocolate', 2),

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Apartado 1.3 TF-IDF (Resuelto)
Obtenemos el TF-IDF utilizando la clase TfidfTransformer

In [7]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer()
X_TFIDF = tfidf_transformer.fit_transform(X_counts)

# Mostramos entonces el número de textos y el número de tokens únicos
print(X_TFIDF.shape)

# X_counts es una matriz dispersa con el TF de cada token en cada texto
# Imprimimos los textos y su correspondientes TF
print(texto)
print(X_TFIDF[0])
print(texto2)
print(X_TFIDF[1])
print(texto3)
print(X_TFIDF[2])

(3, 36)
La asignatura de TGINE es una asignatura del máster de BigData que se estudia en la Universidad de Murcia.
En la asignatura de máster vemos una introducción al procesamiento del lenguaje natural y tecnologías de procesamiento
de información no estructurada.

  (0, 35)	0.12219792805314982
  (0, 33)	0.12219792805314982
  (0, 32)	0.24439585610629963
  (0, 31)	0.09293467399139821
  (0, 30)	0.12219792805314982
  (0, 29)	0.12219792805314982
  (0, 27)	0.12219792805314982
  (0, 25)	0.24439585610629963
  (0, 24)	0.09293467399139821
  (0, 22)	0.12219792805314982
  (0, 21)	0.24439585610629963
  (0, 20)	0.12219792805314982
  (0, 18)	0.12219792805314982
  (0, 16)	0.2788040219741946
  (0, 15)	0.12219792805314982
  (0, 14)	0.12219792805314982
  (0, 10)	0.12219792805314982
  (0, 9)	0.12219792805314982
  (0, 8)	0.09293467399139821
  (0, 7)	0.24439585610629963
  (0, 5)	0.24439585610629963
  (0, 4)	0.5576080439483893
  (0, 2)	0.12219792805314982
  (0, 1)	0.2788040219741946
  (0, 0)	0.122197928053

## Apartado 1.4 BM25 (Resuelto)

Calculamos el BM25 que tiene en cuenta tanto la transformación del TF como la normalización de la longitud del documento.

In [10]:
bm25_transformer = BM25Transformer()
X_BM25 = bm25_transformer.fit_transform(X_counts)

# Mostramos entonces el número de textos y el número de tokens únicos
print(X_BM25.shape)

# X_counts es una matriz dispersa con el TF de cada token en cada texto
# Imprimimos los textos y su correspondientes TF
print(texto)
print(X_BM25[0])
print(texto2)
print(X_BM25[1])
print(texto3)
print(X_BM25[2])

(3, 22)
La asignatura de TGINE es una asignatura del máster de BigData que se estudia en la Universidad de Murcia.
En la asignatura de máster vemos una introducción al procesamiento del lenguaje natural y tecnologías de procesamiento
de información no estructurada.

  (0, 21)	0.23496100930921784
  (0, 19)	0.23496100930921784
  (0, 18)	0.17869390380616346
  (0, 17)	0.23496100930921784
  (0, 14)	0.35619845317922905
  (0, 13)	0.23496100930921784
  (0, 12)	0.35619845317922905
  (0, 11)	0.23496100930921784
  (0, 10)	0.23496100930921784
  (0, 9)	0.23496100930921784
  (0, 8)	0.23496100930921784
  (0, 4)	0.23496100930921784
  (0, 3)	0.23496100930921784
  (0, 1)	0.23496100930921784
  (0, 0)	0.3271702581169214
No me gusta el chocolate ni las fresas
  (0, 7)	0.5773502691896257
  (0, 5)	0.5773502691896257
  (0, 2)	0.5773502691896257
El profesor de la asignatura TGINE es Rafael Valencia García.

  (0, 20)	0.4403620672313486
  (0, 18)	0.3349067026613031
  (0, 16)	0.4403620672313486
  (0, 15)	0.44036

# Apartado 1.5 Desarrollamos un simple buscador con TFIDF y BM25 (Resuelto)

A continuación vamos a procesar los tuits del archivo de la primera sesión "datosEspañol.csv" y calculamos el TF, el TFIDF y el BM25 de manera similar a como se ha hecho anteriormente.

In [26]:
# Calculamos la matriz de TF usando la función fit_transform
count_vect = CountVectorizer(stop_words=stopwords,analyzer='char_wb',ngram_range=(2,4))
X_counts = count_vect.fit_transform(data['tweet'])

# Mostramos entonces el número de textos y el número de tokens únicos
print(X_counts.shape)

# X_counts es una matriz dispersa con el TF de cada token en cada texto
# Imprimimos el primer tuit y su correspondientes TF
print(data['tweet'][0])
print(X_counts[0])

(5959, 75058)
Hoy merendola deliciosa! Latte Macchiato Caramelo con Leche Condensada y Gofre! ???. #yomequedoencasa #todovaasalirbien #undiamenos #actitudpositiva #lattemacchiato #gofre #delicious #strong #stronger #smile…
  (0, 3919)	1
  (0, 37525)	1
  (0, 53991)	1
  (0, 68896)	2
  (0, 3970)	1
  (0, 37673)	1
  (0, 53992)	1
  (0, 3980)	1
  (0, 37674)	1
  (0, 4410)	2
  (0, 46100)	4
  (0, 32964)	2
  (0, 56882)	3
  (0, 32330)	5
  (0, 48129)	3
  (0, 29487)	3
  (0, 51933)	1
  (0, 43048)	3
  (0, 19847)	4
  (0, 4444)	1
  (0, 46246)	1
  (0, 33084)	1
  (0, 57077)	1
  (0, 32423)	1
  (0, 48233)	1
  :	:
  (0, 35873)	1
  (0, 56007)	1
  (0, 48716)	1
  (0, 35963)	1
  (0, 32965)	1
  (0, 52455)	1
  (0, 48727)	1
  (0, 35964)	1
  (0, 61203)	1
  (0, 46373)	1
  (0, 39409)	1
  (0, 34541)	1
  (0, 73508)	1
  (0, 9191)	1
  (0, 61241)	1
  (0, 46462)	1
  (0, 39479)	1
  (0, 43924)	1
  (0, 34542)	1
  (0, 565)	1
  (0, 9193)	1
  (0, 61247)	1
  (0, 46469)	1
  (0, 39489)	1
  (0, 43925)	1


In [27]:
# El vocabulario que forma los tokens del objeto vectorizer se puede obtener de la siguiente manera
# obtenemos el id del token 'buenosdias' que proviene de un hashtag
count_vect.vocabulary_.get('ma')

45658

In [28]:
# Calculamos ahora el TFIDF
X_TFIDF = tfidf_transformer.fit_transform(X_counts)
# Mostramos entonces el número de textos y el número de tokens únicos
print(X_TFIDF.shape)

# X_counts es una matriz dispersa con el TF de cada token en cada texto
# Imprimimos el primer tuit y su correspondientes TF
print(data['tweet'][0])
print(X_TFIDF[0])

# Calculamos también el BM25
X_BM25 = bm25_transformer.fit_transform(X_counts)
# Mostramos entonces el número de textos y el número de tokens únicos
print(X_BM25.shape)

# X_counts es una matriz dispersa con el TF de cada token en cada texto
# Imprimimos el primer tuit y su correspondientes TF
print(data['tweet'][0])
print(X_BM25[0])

(5959, 75058)
Hoy merendola deliciosa! Latte Macchiato Caramelo con Leche Condensada y Gofre! ???. #yomequedoencasa #todovaasalirbien #undiamenos #actitudpositiva #lattemacchiato #gofre #delicious #strong #stronger #smile…
  (0, 73508)	0.038151998348942874
  (0, 69488)	0.01527459292431274
  (0, 69485)	0.015264891882786681
  (0, 69420)	0.01390320178764495
  (0, 68896)	0.026449094151829902
  (0, 67360)	0.04645736145378926
  (0, 67353)	0.04447216063016657
  (0, 67324)	0.0303982876433005
  (0, 67323)	0.03830681221330172
  (0, 66689)	0.024062012172788815
  (0, 66688)	0.017286599186831054
  (0, 66250)	0.04011458481486977
  (0, 66246)	0.030378182236969708
  (0, 66199)	0.012983135166850247
  (0, 65504)	0.015114375472943573
  (0, 65500)	0.010733443552796137
  (0, 65463)	0.008942421505402512
  (0, 65430)	0.06872301294265747
  (0, 65428)	0.05583190501688521
  (0, 65330)	0.02321751134107109
  (0, 64523)	0.06872301294265747
  (0, 64510)	0.04044689336494614
  (0, 64474)	0.02098018738678575
  (0, 644

In [29]:
# Podemos ver que el tamaño de la matriz de textos y el número total de tokens es el mismo tanto
# para TFIDF como para BM25
print(X_TFIDF.shape) # (Number of tweets, Number of unique words)
print(X_BM25.shape)

(5959, 75058)
(5959, 75058)


Realizamos una consulta cualquiera y la metemos en el string "query" para a continuación calcular la similitud del coseno usando el TF-IDF y el BM25

In [15]:
query = "semana santa"

In [16]:
# Transformamos la query a TF-ID y sacamos los resultados de la comparación con la función del coseno
# cosine_similarity
query_vec = count_vect.transform([query]) # Ip -- (n_docs,x), Op -- (n_docs,n_Feats)
query_vec_TFIDF = tfidf_transformer.fit_transform(query_vec)
results = cosine_similarity(X_TFIDF,query_vec_TFIDF).reshape((-1,)) # Op -- (n_docs,1) -- Cosine Sim with each doc

# Immprimimos a continuación los primeros 10 resultados ordenados por la similitud obtenida
for i in results.argsort()[-10:][::-1]:
    print(data.iloc[i,2],"--",results[i],"--",i)

Aunque sean unos momentos difíciles, debemos tomarlo con el mayor positivismo posible! La Semana Santa no es sólo salir a beber y a ver procesiones, es mucho más que eso. Disfrutemos de esta semana de una forma verdadera! Feliz Domingo de Ramos y Semana Santa!!! #YoMeQuedoEnCasa -- 0.5341953468689503 -- 804
Es tiempo de recogimiento ?. Esta Semana Santa no se sale. #SemanaSanta #YoMeQuedoEnCasa #QuedateEnCasa -- 0.4618900044681435 -- 4863
Buen dia amigos espero que estén todos bien!!! Feliz inicio de semana arriba ese ánimo todo lo mejor, que sea una Semana Santa de bendiciones!!! A cuidarse . #QuedateEnCasa -- 0.450087834395828 -- 31
Esta Semana Santa quiero salvar vidas: #YoMeQuedoEnCasa . Si te vas de “vacaciones”, ¿cuántas muertes caerán sobre tus espaldas la semana que viene...?. #QuedateEnTuCasa ?? -- 0.40809989875165054 -- 1372
¡Feliz Semana Santa! Aunque sea en casa ?. #AyudaADomicilio #Mayores #QuéDateEnCasa #Santander #Cantabria -- 0.4044128930022517 -- 1967
La procesión de c

In [17]:
# Obtenemos ahora los resultados usando el BM25
query_vec = count_vect.transform([query]) # Ip -- (n_docs,x), Op -- (n_docs,n_Feats)
query_vec_BM25 = bm25_transformer.fit_transform(query_vec)
results = cosine_similarity(X_BM25,query_vec_BM25).reshape((-1,)) # Op -- (n_docs,1) -- Cosine Sim with each doc

# Immprimimos a continuación los primeros 10 resultados ordenados por la similitud obtenida
for i in results.argsort()[-10:][::-1]:
    print(data.iloc[i,2],"--",results[i],"--",i)

Es tiempo de recogimiento ?. Esta Semana Santa no se sale. #SemanaSanta #YoMeQuedoEnCasa #QuedateEnCasa -- 0.46189000446814343 -- 4863
¡Feliz Semana Santa! Aunque sea en casa ?. #AyudaADomicilio #Mayores #QuéDateEnCasa #Santander #Cantabria -- 0.4044128930022516 -- 1967
La procesión de carros de compra que estoy viendo desde mi balcón sirve como Semana Santa? #YoMeQuedoEnCasa -- 0.3838919966764931 -- 156
Aunque sean unos momentos difíciles, debemos tomarlo con el mayor positivismo posible! La Semana Santa no es sólo salir a beber y a ver procesiones, es mucho más que eso. Disfrutemos de esta semana de una forma verdadera! Feliz Domingo de Ramos y Semana Santa!!! #YoMeQuedoEnCasa -- 0.3810789571546703 -- 804
Buen dia amigos espero que estén todos bien!!! Feliz inicio de semana arriba ese ánimo todo lo mejor, que sea una Semana Santa de bendiciones!!! A cuidarse . #QuedateEnCasa -- 0.38098584294080773 -- 31
Último día de teletrabajo hasta después de Semana Santa ???. Gracias equipo TeSer

## Apartado 1.5 Calculamos la similitud de varios textos con TF-IDF y BM25 (Resuelto)

Vamos a calcular la similitud de el primer texto con respecto a los demás usando la similitud del coseno.

In [30]:
# Definimos un conjunto de textos
textos=['El procesamiento del lenguaje natural (PLN o NLP) es un campo dentro de la inteligencia artificial y la lingüística aplicada que estudia las interacciones mediante uso del lenguaje natural entre los seres humanos y las máquinas. \
Más concretamente se centra en el procesamiento de las comunicaciones humanas, dividiéndolas en partes, e identificando los elementos más relevantes del mensaje.\
Con la Comprensión y Generación de Lenguaje Natural, busca que las máquinas consigan entender, interpretar y manipular el lenguaje humano.'
, 'El procesamiento del lenguaje natural (NLP, por sus siglas en inglés) es una rama de la inteligencia artificial que ayuda a las computadoras a entender, interpretar y manipular el lenguaje humano. \
NLP toma elementos prestados de muchas disciplinas, incluyendo la ciencia de la computación y la lingüística computacional, en su afán por cerrar la brecha entre la comunicación humana y el entendimiento de las computadoras."""], """El procesamiento del lenguaje natural (PLN o NLP) es un campo dentro de la inteligencia artificial y la lingüística aplicada que estudia las interacciones mediante uso del lenguaje natural entre los seres humanos y las máquinas. Más concretamente se centra en el procesamiento de las comunicaciones humanas, dividiéndolas en partes, e identificando los elementos más relevantes del mensaje. Con la Comprensión y Generación de Lenguaje Natural, busca que las máquinas consigan entender, interpretar y manipular el lenguaje humano.'
, 'La lingüística computacional es un campo interdisciplinario que se ocupa del desarrollo de formalismos del funcionamiento del lenguaje natural, tales que puedan ser transformados en programas ejecutables para un ordenador. \
Dicho desarrollo se sitúa entre el modelado basado en reglas y el modelado estadístico del lenguaje natural desde una perspectiva computacional, y en él participan lingüistas e informáticos especializados en inteligencia artificial, psicólogos cognoscitivos y expertos en lógica, entre otros.'
, 'El aprendizaje automático es un tipo de inteligencia artificial (AI) que proporciona a las computadoras la capacidad de aprender, sin ser programadas explícitamente. El aprendizaje automático se centra en el desarrollo de programas informáticos que pueden cambiar cuando se exponen a nuevos datos.'
, 'El  aprendizaje profundo es un tema que cada vez adquiere mayor relevancia en el campo de la inteligencia artificial (IA). Siendo una subcategoría del aprendizaje automático, el aprendizaje profundo trata del uso de redes neuronales para mejorar cosas tales como el reconocimiento de voz, la visión por ordenador y el procesamiento del lenguaje natural. \
Rápidamente se está convirtiendo en uno de los campos más solicitados en informática. \
En los últimos años, el aprendizaje profundo ha ayudado a lograr avances en áreas tan diversas como la percepción de objetos, el procesamiento del lenguaje natural y el reconocimiento de voz (todas ellas áreas especialmente complejas para los investigadores en IA).'
]

# Calculamos la similitud usando TFIDF
count_vect = CountVectorizer(stop_words=stopwords)
X_counts = count_vect.fit_transform(textos)

# Calculamos ahora el TFIDF
tfidf_transformer = TfidfTransformer()
X_TFIDF = tfidf_transformer.fit_transform(X_counts)

# Calculamos también el BM25
BM25_transformer = BM25Transformer(k=1.2,b=0.75)
X_BM25 = BM25_transformer.fit_transform(X_counts)

# Calculamos la similitud de los documentos con el coseno para TFIDF
results = cosine_similarity(X_TFIDF[1::],X_TFIDF[0]).reshape((-1,)) # Op -- (n_docs,1) -- Cosine Sim with each doc
print(results)

# Calculamos la similitud de los textos con el coseno para BM25
results = cosine_similarity(X_BM25[1::],X_TFIDF[0]).reshape((-1,)) # Op -- (n_docs,1) -- Cosine Sim with each doc
print(results)



[0.82218193 0.16800859 0.03486424 0.15382285]
[0.7465174  0.1372938  0.03771061 0.1419987 ]


##1.6 Cambiando a character n-grams
Podemos usar en vez de palabras (words), character n-grams para crear el vocabulario. Lo vemos con el primer ejemplo

In [31]:
texto = """La asignatura de TGINE es una asignatura del máster de BigData que se estudia en la Universidad de Murcia.
En la asignatura de máster vemos una introducción al procesamiento del lenguaje natural y tecnologías de procesamiento
de información no estructurada.
"""
texto2 = "No me gusta el chocolate ni las fresas"

texto3 = """El profesor de la asignatura TGINE es Rafael Valencia García.
"""
# Calculamos la matriz de TF usando la función fit_transform
count_vect = CountVectorizer()
X_counts = count_vect.fit_transform([texto,texto2,texto3])

# Mostramos entonces el número de textos y el número de tokens únicos
print(X_counts.shape)

# X_counts es una matriz dispersa con el TF de cada token en cada texto
# Imprimimos los textos y su correspondientes TF
print(texto)
print(X_counts[0])
print(texto2)
print(X_counts[1])
print(texto3)
print(X_counts[2])

#Los tokens de todo el vocabulario se representan con ids que hacen referencia a cada token.
print("Mostramos los items del diccionario")
print(count_vect.vocabulary_.items())
print("Tamaño vocabulario:", str(len(count_vect.vocabulary_.items())))

#Mostramos el código de una palabra determinada
#hay que tener en cuenta que todos los tokens se guardan en minúsculas
palabra_a_buscar="TGINE"
print("Código de la palabra", palabra_a_buscar, "es:", count_vect.vocabulary_.get(palabra_a_buscar.lower()))

(3, 36)
La asignatura de TGINE es una asignatura del máster de BigData que se estudia en la Universidad de Murcia.
En la asignatura de máster vemos una introducción al procesamiento del lenguaje natural y tecnologías de procesamiento
de información no estructurada.

  (0, 16)	3
  (0, 1)	3
  (0, 4)	6
  (0, 31)	1
  (0, 8)	1
  (0, 32)	2
  (0, 5)	2
  (0, 21)	2
  (0, 2)	1
  (0, 27)	1
  (0, 29)	1
  (0, 10)	1
  (0, 7)	2
  (0, 33)	1
  (0, 20)	1
  (0, 35)	1
  (0, 15)	1
  (0, 0)	1
  (0, 25)	2
  (0, 18)	1
  (0, 22)	1
  (0, 30)	1
  (0, 14)	1
  (0, 24)	1
  (0, 9)	1
No me gusta el chocolate ni las fresas
  (0, 24)	1
  (0, 19)	1
  (0, 13)	1
  (0, 6)	1
  (0, 3)	1
  (0, 23)	1
  (0, 17)	1
  (0, 11)	1
El profesor de la asignatura TGINE es Rafael Valencia García.

  (0, 16)	1
  (0, 1)	1
  (0, 4)	1
  (0, 31)	1
  (0, 8)	1
  (0, 6)	1
  (0, 26)	1
  (0, 28)	1
  (0, 34)	1
  (0, 12)	1
Mostramos los items del diccionario
dict_items([('la', 16), ('asignatura', 1), ('de', 4), ('tgine', 31), ('es', 8), ('una', 32), 